# Facebook Page-Level Insights 自動化數據抓取工具

從 Facebook Graph API 獲取粉絲專頁的頁面層級數據，並彙整至 Google Sheets。

**數據包含**：每日頁面層級指標 (觸及/互動/影片觀看等) 和終身指標 (粉絲數/追蹤者數)

## 1. 套件安裝與匯入

In [ ]:
# 安裝必要套件
!pip install requests pandas gspread google-auth google-auth-oauthlib google-auth-httplib2

# 匯入必要函式庫
import requests
import pandas as pd
import gspread
from google.oauth2 import service_account
import json
from datetime import datetime, timedelta
import time
import os
from typing import List, Dict, Any, Optional

## 2. 設定檔與憑證

In [ ]:
# 掛載 Google Drive (用於存取憑證檔案)
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Facebook API 設定
FACEBOOK_CONFIG = {
    'app_id': '1085898272974442',
    'page_id': '103640919705348',
    'access_token': 'EAAPbnmTSpmoBPsBfJHKn3AZAHpCZC2XvkyYvZAcKmZCQHPnPL44i8yevD1PAxSGjaRRl87RFZB79vTKPGKAFdbT35HfZAcZApp5j76f6hkIXCsO6Sgmi06H7mbkOELkn3gfqLU2UKSDaDMfs5oeNyfBMfVpmG4GSlb1WF9GJ3pluweVV0mb2Jp79bWfkcvYZAx4eKNJtZApYZD',
    'api_version': 'v23.0'
}

# Google Sheets 設定
GOOGLE_SHEETS_CONFIG = {
    'credentials_path': '/content/drive/MyDrive/Colab Notebooks/gemini-api-reports-3a9837dee55c.json',
    'spreadsheet_name': 'Faceboook Insights Metrics_Data Warehouse',
    'worksheet_name': 'raw_page_data'  # 新工作表用於頁面層級數據
}

# 頁面層級每日 Insights 指標
# 根據 Facebook API 文檔，以下為目前可用的每日頁面層級指標
PAGE_INSIGHTS_METRICS = [
    'page_impressions_unique',          # 頁面觸及人數
    'page_post_engagements',            # 貼文互動數
    'page_video_views',                 # 影片觀看次數
    'page_actions_post_reactions_total' # 貼文心情總數
]

# 終身指標 (非每日，單一數值)
LIFETIME_METRICS_FIELDS = [
    'fan_count',        # 粉絲總數
    'followers_count'   # 追蹤者總數
]

print("設定載入完成")
print(f"目標 Page ID: {FACEBOOK_CONFIG['page_id']}")
print(f"API 版本: {FACEBOOK_CONFIG['api_version']}")
print(f"每日指標數量: {len(PAGE_INSIGHTS_METRICS)} 項")
print(f"終身指標數量: {len(LIFETIME_METRICS_FIELDS)} 項")
print(f"\n注意：許多 page_* 指標已被 Facebook API 棄用")
print(f"請參考 '合併文章總集.md' 以確保使用最新可用指標")

## 3. 核心功能函式

In [ ]:
def test_facebook_api_connection(config: Dict[str, str]) -> bool:
    """
    測試 Facebook API 連接是否正常
    """
    try:
        url = f"https://graph.facebook.com/{config['api_version']}/{config['page_id']}"
        params = {
            'access_token': config['access_token'],
            'fields': 'id,name,fan_count,followers_count'
        }

        response = requests.get(url, params=params)
        response.raise_for_status()

        data = response.json()
        print(f"✓ API 連接成功")
        print(f"  Page ID: {data.get('id')}")
        print(f"  Page Name: {data.get('name')}")
        print(f"  粉絲數: {data.get('fan_count', 'N/A')}")
        print(f"  追蹤者數: {data.get('followers_count', 'N/A')}")
        return True

    except requests.exceptions.RequestException as e:
        print(f"✗ API 連接失敗: {e}")
        return False
    except Exception as e:
        print(f"✗ 未預期的錯誤: {e}")
        return False


def fetch_page_insights(config: Dict[str, str], metrics: List[str], since: str, until: str) -> Optional[Dict]:
    """
    從 Facebook API 獲取頁面層級的每日 Insights 數據

    Args:
        config: Facebook API 設定
        metrics: 要獲取的指標列表
        since: 開始日期 (YYYY-MM-DD)
        until: 結束日期 (YYYY-MM-DD)

    Returns:
        以日期為鍵的字典，每個日期包含該日所有指標數值
        格式: {'2025-10-01': {'metric1': value1, 'metric2': value2}, ...}
    """
    try:
        url = f"https://graph.facebook.com/{config['api_version']}/{config['page_id']}/insights"

        # 轉換日期為 Unix timestamp
        since_ts = int(datetime.strptime(since, '%Y-%m-%d').timestamp())
        until_ts = int(datetime.strptime(until, '%Y-%m-%d').timestamp())

        params = {
            'access_token': config['access_token'],
            'metric': ','.join(metrics),
            'period': 'day',
            'since': since_ts,
            'until': until_ts
        }

        response = requests.get(url, params=params)
        response.raise_for_status()

        data = response.json()
        insights_data = data.get('data', [])

        # 重組數據：以日期為主鍵
        daily_data = {}

        for metric_entry in insights_data:
            metric_name = metric_entry.get('name')
            values = metric_entry.get('values', [])

            for value_entry in values:
                end_time = value_entry.get('end_time', '')
                if end_time:
                    # 轉換為日期字串 (YYYY-MM-DD)
                    date_str = end_time.split('T')[0]

                    if date_str not in daily_data:
                        daily_data[date_str] = {}

                    daily_data[date_str][metric_name] = value_entry.get('value', 0)

        print(f"✓ 成功獲取 {len(daily_data)} 天的頁面 Insights 數據")
        return daily_data

    except requests.exceptions.RequestException as e:
        print(f"✗ 頁面 Insights API 請求失敗: {e}")
        if hasattr(e, 'response') and e.response is not None:
            try:
                error_detail = e.response.json()
                print(f"錯誤詳情: {error_detail}")
            except:
                pass
        return None
    except Exception as e:
        print(f"✗ 未預期的錯誤: {e}")
        return None


def fetch_lifetime_metrics(config: Dict[str, str], fields: List[str]) -> Optional[Dict]:
    """
    從 Facebook API 獲取頁面的終身指標

    Args:
        config: Facebook API 設定
        fields: 要獲取的欄位列表

    Returns:
        包含終身指標的字典
    """
    try:
        url = f"https://graph.facebook.com/{config['api_version']}/{config['page_id']}"
        params = {
            'access_token': config['access_token'],
            'fields': ','.join(fields)
        }

        response = requests.get(url, params=params)
        response.raise_for_status()

        data = response.json()
        print(f"✓ 成功獲取終身指標")
        return data

    except requests.exceptions.RequestException as e:
        print(f"✗ 終身指標 API 請求失敗: {e}")
        if hasattr(e, 'response') and e.response is not None:
            try:
                error_detail = e.response.json()
                print(f"錯誤詳情: {error_detail}")
            except:
                pass
        return None
    except Exception as e:
        print(f"✗ 未預期的錯誤: {e}")
        return None

In [ ]:
def process_page_data(daily_data: Dict, lifetime_data: Dict, config: Dict[str, str], fetch_date: str) -> pd.DataFrame:
    """
    處理頁面數據，轉換為結構化的 DataFrame

    Args:
        daily_data: 每日指標數據 (以日期為鍵的字典)
        lifetime_data: 終身指標數據
        config: Facebook API 設定
        fetch_date: 數據抓取日期

    Returns:
        結構化的 DataFrame，每列代表一天的數據
    """
    processed_records = []

    for data_date, metrics in daily_data.items():
        record = {
            'fetch_date': fetch_date,
            'data_date': data_date,
            'page_id': config['page_id']
        }

        # 加入每日指標
        for metric_name, value in metrics.items():
            # 處理 page_actions_post_reactions_total（字典類型）
            if metric_name == 'page_actions_post_reactions_total' and isinstance(value, dict):
                # 展開為個別反應欄位
                record['reactions_like'] = value.get('like', 0)
                record['reactions_love'] = value.get('love', 0)
                record['reactions_wow'] = value.get('wow', 0)
                record['reactions_haha'] = value.get('haha', 0)
                record['reactions_sorry'] = value.get('sorry', 0)
                record['reactions_anger'] = value.get('anger', 0)
                # 計算總數
                record['reactions_total'] = sum(value.values())
            else:
                record[metric_name] = value

        # 加入終身指標
        for field in LIFETIME_METRICS_FIELDS:
            record[field] = lifetime_data.get(field, 0)

        processed_records.append(record)

    return pd.DataFrame(processed_records)


def validate_page_dataframe(df: pd.DataFrame) -> bool:
    """
    驗證頁面數據 DataFrame 的結構是否正確
    """
    required_columns = ['fetch_date', 'data_date', 'page_id']

    if df.empty:
        print("⚠️ DataFrame 為空")
        return False

    missing_columns = [col for col in required_columns if col not in df.columns]
    if missing_columns:
        print(f"⚠️ 缺少必要欄位: {missing_columns}")
        return False

    print(f"✓ DataFrame 驗證通過 ({len(df)} 天的數據)")
    return True

In [ ]:
def setup_google_sheets_client(credentials_path: str) -> Optional[gspread.Client]:
    """
    設定 Google Sheets 客戶端
    """
    try:
        if not os.path.exists(credentials_path):
            print(f"✗ 找不到憑證檔案: {credentials_path}")
            return None

        # 設定 Google Sheets API 的權限範圍
        scope = [
            'https://www.googleapis.com/auth/spreadsheets',
            'https://www.googleapis.com/auth/drive'
        ]

        # 載入服務帳戶憑證
        credentials = service_account.Credentials.from_service_account_file(
            credentials_path, scopes=scope)

        # 建立 gspread 客戶端
        client = gspread.authorize(credentials)

        print("✓ Google Sheets 客戶端設定成功")
        return client

    except Exception as e:
        print(f"✗ Google Sheets 客戶端設定失敗: {e}")
        return None


def write_data_to_google_sheets(client: gspread.Client, config: Dict[str, str], df: pd.DataFrame) -> bool:
    """
    將 DataFrame 寫入 Google Sheets，確保欄位順序與名稱正確，並避免重複記錄
    """
    try:
        # 開啟試算表
        spreadsheet = client.open(config['spreadsheet_name'])
        
        # 嘗試開啟工作表，如果不存在則建立
        try:
            worksheet = spreadsheet.worksheet(config['worksheet_name'])
        except gspread.exceptions.WorksheetNotFound:
            print(f"⚠️ 工作表 '{config['worksheet_name']}' 不存在，建立新工作表...")
            worksheet = spreadsheet.add_worksheet(title=config['worksheet_name'], rows=1000, cols=20)

        # 定義正確的欄位順序
        expected_columns = [
            'fetch_date',
            'data_date',
            'page_id',
            'page_impressions_unique',
            'page_post_engagements',
            'page_video_views',
            'reactions_like',
            'reactions_love',
            'reactions_wow',
            'reactions_haha',
            'reactions_sorry',
            'reactions_anger',
            'reactions_total',
            'fan_count',
            'followers_count'
        ]

        # 確保 DataFrame 包含所有必要欄位（缺少的填 0 或空字串）
        for col in expected_columns:
            if col not in df.columns:
                if col in ['fetch_date', 'data_date', 'page_id']:
                    df[col] = ''
                else:
                    df[col] = 0

        # 按照正確順序重新排列欄位
        df = df[expected_columns]

        # 檢查工作表是否為空或標題列不正確
        existing_data = worksheet.get_all_values()

        if not existing_data or not existing_data[0]:
            # 工作表為空，寫入標題列
            worksheet.append_row(expected_columns)
            print("✓ 已寫入標題列")
            existing_data = [expected_columns]
        else:
            # 檢查現有標題列是否正確
            existing_headers = existing_data[0]
            if existing_headers != expected_columns:
                print(f"⚠️  警告：工作表標題列與預期不符")
                print(f"   現有標題: {existing_headers[:3]}... ({len(existing_headers)} 個欄位)")
                print(f"   預期標題: {expected_columns[:3]}... ({len(expected_columns)} 個欄位)")
                print("   自動更新標題列...")
                worksheet.delete_rows(1)
                worksheet.insert_row(expected_columns, 1)
                print("✓ 已更新標題列")
                existing_data = worksheet.get_all_values()

        # 檢查重複記錄
        # 建立現有記錄的 page_id + data_date 組合集合
        existing_records = set()
        if len(existing_data) > 1:  # 有數據行（不只標題）
            headers = existing_data[0]
            try:
                page_id_idx = headers.index('page_id')
                data_date_idx = headers.index('data_date')

                for row in existing_data[1:]:
                    if len(row) > max(page_id_idx, data_date_idx):
                        key = f"{row[page_id_idx]}_{row[data_date_idx]}"
                        existing_records.add(key)

                print(f"✓ 已載入 {len(existing_records)} 筆現有記錄")
            except ValueError as e:
                print(f"⚠️  無法找到必要欄位索引: {e}")

        # 過濾出新記錄
        new_rows = []
        duplicate_count = 0

        for idx, row in df.iterrows():
            key = f"{row['page_id']}_{row['data_date']}"
            if key not in existing_records:
                new_rows.append(row.tolist())
            else:
                duplicate_count += 1

        print(f"\n數據去重結果:")
        print(f"  總記錄數: {len(df)}")
        print(f"  新記錄: {len(new_rows)}")
        print(f"  重複記錄（已跳過）: {duplicate_count}")

        # 批次寫入新數據
        if new_rows:
            worksheet.append_rows(new_rows)
            print(f"\n✓ 已寫入 {len(new_rows)} 筆新數據到 Google Sheets")
            print(f"   欄位數: {len(expected_columns)}")
            return True
        else:
            print("\n⚠️  無新數據需要寫入（所有記錄已存在）")
            return True

    except gspread.exceptions.SpreadsheetNotFound:
        print(f"✗ 找不到試算表: {config['spreadsheet_name']}")
        return False
    except Exception as e:
        print(f"✗ 寫入 Google Sheets 失敗: {e}")
        import traceback
        traceback.print_exc()
        return False

## 4. 系統測試與連接驗證

In [ ]:
# 測試 Facebook API 連接
print("=== Facebook API 連接測試 ===")
api_connected = test_facebook_api_connection(FACEBOOK_CONFIG)

if not api_connected:
    print("\n⚠️ API 連接失敗，請檢查 Access Token 是否有效")
    print("建議檢查事項:")
    print("1. Access Token 是否已過期")
    print("2. App 是否有適當的權限 (pages_read_engagement, read_insights)")
    print("3. Page ID 是否正確")

# 測試 Google Sheets 連接
print("\n=== Google Sheets 連接測試 ===")
sheets_client = setup_google_sheets_client(GOOGLE_SHEETS_CONFIG['credentials_path'])

if not sheets_client:
    print("\n⚠️ Google Sheets 連接失敗")
    print("建議檢查事項:")
    print("1. 憑證檔案路徑是否正確")
    print("2. 服務帳戶是否有試算表的編輯權限")
    print("3. Drive 是否已掛載")

In [ ]:
# 測試頁面 Insights 獲取功能
print("\n=== 頁面 Insights 獲取測試 ===")

if api_connected:
    # 測試獲取最近 3 天的數據
    test_until = (datetime.now() - timedelta(days=1)).strftime('%Y-%m-%d')
    test_since = (datetime.now() - timedelta(days=3)).strftime('%Y-%m-%d')

    print(f"測試日期範圍: {test_since} 到 {test_until}")

    # 測試每日指標
    print(f"\n測試獲取每日頁面 Insights...")
    test_daily_data = fetch_page_insights(FACEBOOK_CONFIG, PAGE_INSIGHTS_METRICS, test_since, test_until)

    if test_daily_data:
        print(f"\n可用的每日數據:")
        for date, metrics in list(test_daily_data.items())[:2]:
            print(f"\n  日期: {date}")
            for metric_name, value in metrics.items():
                print(f"    - {metric_name}: {value}")
    else:
        print("⚠️ 無法獲取每日 Insights 數據")

    # 測試終身指標
    print(f"\n測試獲取終身指標...")
    test_lifetime_data = fetch_lifetime_metrics(FACEBOOK_CONFIG, LIFETIME_METRICS_FIELDS)

    if test_lifetime_data:
        print(f"\n終身指標:")
        for field in LIFETIME_METRICS_FIELDS:
            print(f"  - {field}: {test_lifetime_data.get(field, 'N/A')}")
    else:
        print("⚠️ 無法獲取終身指標")
else:
    print("跳過 Insights 獲取測試 (API 連接失敗)")

## 5. 主要執行程序

In [ ]:
def main_page_collection(since_date: str = None, until_date: str = None) -> bool:
    """
    主要的頁面層級數據蒐集流程

    Args:
        since_date: 開始日期 (預設為90天前)
        until_date: 結束日期 (預設為昨天)

    Returns:
        是否成功
    """
    # 設定日期範圍 - 預設為最近 90 天
    if until_date is None:
        until_date = (datetime.now() - timedelta(days=1)).strftime('%Y-%m-%d')
    if since_date is None:
        since_date = (datetime.now() - timedelta(days=90)).strftime('%Y-%m-%d')

    fetch_date = datetime.now().strftime('%Y-%m-%d')

    print(f"=== 開始頁面層級數據蒐集 ===")
    print(f"日期範圍: {since_date} 到 {until_date}")
    print(f"執行日期: {fetch_date}")

    # 檢查前置條件
    if not api_connected:
        print("✗ Facebook API 未連接")
        return False

    if not sheets_client:
        print("✗ Google Sheets 未連接")
        return False

    # 步驟 1: 獲取終身指標
    print(f"\n步驟 1: 獲取終身指標")
    lifetime_data = fetch_lifetime_metrics(FACEBOOK_CONFIG, LIFETIME_METRICS_FIELDS)

    if lifetime_data is None:
        print("✗ 無法獲取終身指標")
        return False

    # 步驟 2: 獲取每日頁面 Insights
    print(f"\n步驟 2: 獲取每日頁面 Insights")
    daily_data = fetch_page_insights(FACEBOOK_CONFIG, PAGE_INSIGHTS_METRICS, since_date, until_date)

    if daily_data is None:
        print("✗ 無法獲取每日 Insights")
        return False

    if not daily_data:
        print("⚠️ 該日期範圍內無數據")
        return False

    # 步驟 3: 處理數據
    print(f"\n步驟 3: 處理數據")
    df = process_page_data(daily_data, lifetime_data, FACEBOOK_CONFIG, fetch_date)

    if not validate_page_dataframe(df):
        print("✗ 數據驗證失敗")
        return False

    # 顯示數據預覽
    print(f"\n=== 數據預覽 ===")
    print(f"欄位: {list(df.columns)}")
    print(f"\n前 5 天數據:")
    print(df.head())

    print(f"\n數據摘要:")
    print(f"- 總天數: {len(df)}")
    print(f"- 日期範圍: {df['data_date'].min()} 到 {df['data_date'].max()}")
    if 'page_impressions_unique' in df.columns:
        print(f"- 平均每日觸及: {df['page_impressions_unique'].mean():.0f}")
    if 'page_post_engagements' in df.columns:
        print(f"- 總互動數: {df['page_post_engagements'].sum():.0f}")

    # 步驟 4: 寫入 Google Sheets
    print(f"\n步驟 4: 寫入 Google Sheets")
    success = write_data_to_google_sheets(sheets_client, GOOGLE_SHEETS_CONFIG, df)

    if success:
        print(f"\n🎉 頁面層級數據蒐集完成!")
        print(f"已將數據寫入 Google Sheets")
        return True
    else:
        print(f"\n❌ 頁面層級數據蒐集失敗")
        return False


# 執行主程序
if 'api_connected' in locals() and 'sheets_client' in locals():
    # 預設收集最近 90 天的頁面數據
    success = main_page_collection()
else:
    print("請先執行上述所有測試單元格")

## 6. 進階功能：自訂日期範圍執行

In [ ]:
# 自訂日期範圍執行範例
# 取消註解以下程式碼以執行自訂日期範圍的數據蒐集

# custom_since = '2025-09-01'
# custom_until = '2025-09-30'
# success = main_page_collection(since_date=custom_since, until_date=custom_until)